### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Dados Reais do Sistema

In [ ]:
def plot_data(u,y):
    plt.figure(figsize=(20, 6))
    plt.plot(u, label='u(t)', linestyle='-', color = 'r')
    plt.plot(y, label='y(t)', linestyle='-', color = 'k')
    plt.xlabel('Tempo (s)')
    plt.ylabel('Amplitude')
    plt.title('Resposta do Sistema ao longo do tempo')
    plt.legend()
    plt.grid(True)
    plt.show()

### FROLS (Forward-Regression Orthogonal Least Squares) para identificar sistemas NARMAX (Nonlinear AutoRegressive Moving-Average with eXogenous inputs). Esses sistemas são representados por uma equação geral da forma:

\begin{equation*}
y(t) = f(y(t-1), y(t-2), ..., y(t-na), u(t-1), u(t-2), ..., u(t-nb)) + e(t)
\end{equation*}

### Onde:
- y(t) é a saída do sistema no instante de tempo t.
- u(t) é a entrada do sistema no instante de tempo t.
- na e nb são as ordens dos polinômios auto-regressivos e de média móvel, respectivamente.
- f() é uma função não linear que representa a dinâmica do sistema.
- e(t) é um termo de erro.

In [ ]:
def frols(u, y):
    """
    Identifica um modelo NARMAX usando o algoritmo FROLS (Forward-Regression Orthogonal Least Squares).

    Args:
        u (array): Um array numpy contendo os dados de entrada (u).
        y (array): Um array numpy contendo os dados de saída (y).

    Returns:
        tuple: Uma tupla contendo informações sobre o modelo identificado:
            - na (int): A ordem dos termos auto-regressivos selecionados.
            - nb (int): A ordem dos termos de média móvel selecionados.
            - beta (array): Os coeficientes do modelo identificado.
            - selected_terms (list): Uma lista de termos selecionados (por exemplo, ['y(t-1)', 'u(t-2)']).
            - X (array): A matriz de regressores usada para a identificação.
    """

    na, nb = 1, 1 # ordens dos termos auto-regressivos e de média móvel
    prev_error = float('inf')  # erro anterior como infinito
    
    max_na = len(y) // 2  # Ordem máxima dos termos auto-regressivos
    max_nb = len(u) // 2  # Ordem máxima dos termos de média móvel
    selected_terms = []  # Lista para armazenar os termos selecionados

    X = np.empty((len(u), 0))  # matriz de regressores como vazia
    beta = np.array([])  # vetor de coeficientes como vazio
    # Continue o loop enquanto as ordens máximas não forem excedidas ou enquanto houver redução no erro
    while (na <= max_na or nb <= max_nb) and prev_error > 0:
        # print(na)
        if na <= max_na:
            # Adicione um termo auto-regressivo
            X_na = np.column_stack([X, np.roll(y, na)])
            # print(X_na)
            error_na = np.sum((y - np.dot(X_na, np.linalg.lstsq(X_na, y, rcond=None)[0]))**2)
            # error_na = 1 - (prev_error / error_na)
            
        if nb <= max_nb:
            # Adicione um termo de média móvel
            X_nb = np.column_stack([X, np.roll(u, nb)])
            # print(X_nb)
            error_nb = np.sum((y - np.dot(X_nb, np.linalg.lstsq(X_nb, y, rcond=None)[0]))**2)
            # error_nb = 1 - (prev_error / error_nb)
        
        if na <= max_na and error_na < prev_error:
            # Aceite o termo auto-regressivo
            X = X_na
            selected_terms.append(f'y(t-{na + 1})')
            # na += 1
            prev_error = error_na
            beta = np.linalg.lstsq(X, y, rcond=None)[0]
        elif nb <= max_nb and error_nb < prev_error:
            # Aceite o termo de média móvel
            X = X_nb
            selected_terms.append(f'u(t-{nb + 1})')
            # nb += 1
            prev_error = error_nb
            beta = np.linalg.lstsq(X, y, rcond=None)[0]
        else:
            break
            
        na += 1
        nb += 1
    return beta, selected_terms, X

# Exemplo de uso:
# na, nb, beta, selected_terms, X = frols(u, y)

### Base de dados para Benchmarks

In [ ]:
tanque     = pd.read_csv('tanque.csv')
silver_box = pd.read_csv('SNLS80mV.csv')
ballbeam   = np.loadtxt('ballbeam.dat')
Liquid_saturated = np.loadtxt('Liquid-saturated.dat')
robot_arm  = np.loadtxt('robot_arm.dat')

### Cascaded Tanks

In [ ]:
tanque     = pd.read_csv('tanque.csv')
u_tanque = tanque['uVal'].values
y_tanque = tanque['yVal'].values
# plot_data(u_tanque, y_tanque)

In [ ]:
beta_1, selected_terms_1, X_1 = frols(u_tanque, y_tanque)
y_pred_1 = np.dot(X_1, beta_1)

plt.figure(figsize=(20, 6))
plt.plot(y_tanque, label='Real', linestyle='-', color = 'r')
plt.plot(y_pred_1, label='Predito', linestyle='-',color = 'k')
plt.xlabel('Tempo')
plt.ylabel('yVal')
plt.title('Comparação entre Dados Reais e Preditos')
plt.legend()
plt.grid(True)
plt.show()

### Silverbox System

In [ ]:
u_sb = silver_box['V1'].values
y_sb = silver_box['V2'].values
# plot_data(u_sb, y_sb)

In [ ]:
beta_2, selected_terms_2, X_2 = frols(u_sb, y_sb)
y_pred_2 = np.dot(X_2, beta_2)

plt.figure(figsize=(20, 6))
plt.plot(y_sb, label='Real', linestyle='-', color = 'r')
plt.plot(y_pred_2, label='Predito', linestyle='-',color = 'k')
plt.xlabel('Tempo')
plt.ylabel('yVal')
plt.title('Comparação entre Dados Reais e Preditos')
plt.legend()
plt.grid(True)
plt.show()

### Liquid-saturated steam heat exchanger

In [ ]:
u_liquid_saturated = Liquid_saturated[:, 1]
y_liquid_saturated = Liquid_saturated[:, 2]
# plot_data(u_liquid_saturated, y_liquid_saturated)

In [ ]:
beta_3, selected_terms_3, X_3 = frols(u_liquid_saturated, y_liquid_saturated)
y_pred_3 = np.dot(X_3, beta_3)

plt.figure(figsize=(20, 6))
plt.plot(y_liquid_saturated, label='Real', linestyle='-', color = 'r')
plt.plot(y_pred_3, label='Predito', linestyle='-',color = 'k')
plt.xlabel('Tempo')
plt.ylabel('yVal')
plt.title('Comparação entre Dados Reais e Preditos')
plt.legend()
plt.grid(True)
plt.show()

###  Data of the ball-and-beam setup in STADIUS

In [ ]:
u_ballbeam = ballbeam[:,0]
y_ballbeam = ballbeam[:,1]
# plot_data(u_ballbeam, y_ballbeam)

In [ ]:
beta_4, selected_terms_4, X_4 = frols(u_ballbeam, y_ballbeam)
y_pred_4 = np.dot(X_4, beta_4)

plt.figure(figsize=(20, 6))
plt.plot(y_ballbeam, label='Real', linestyle='-', color = 'r')
plt.plot(y_pred_4, label='Predito', linestyle='-',color = 'k')
plt.xlabel('Tempo')
plt.ylabel('yVal')
plt.title('Comparação entre Dados Reais e Preditos')
plt.legend()
plt.grid(True)
plt.show()

### Data from a flexible robot arm 

In [ ]:
u_robot_arm = robot_arm[:,0]
y_robot_arm = robot_arm[:,1]
# plot_data(u_robot_arm, y_robot_arm)

In [ ]:
beta_6, selected_terms_6, X_6 = frols(u_robot_arm, y_robot_arm)
y_pred_6 = np.dot(X_6, beta_6)

plt.figure(figsize=(20, 6))
plt.plot(y_robot_arm, label='Real', linestyle='-', color = 'r')
plt.plot(y_pred_6, label='Predito', linestyle='-',color = 'k')
plt.xlabel('Tempo')
plt.ylabel('Amplitude')
plt.title('Comparação entre Dados Reais e Preditos')
plt.legend()
plt.grid(True)
plt.show()